## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-14-16-49-56 +0000,wapo,Witness describes terror inside Brown Universi...,https://www.washingtonpost.com
1,2025-12-14-16-49-56 +0000,wapo,Live updates: Police detain person of interest...,https://www.washingtonpost.com/nation/2025/12/...
2,2025-12-14-16-48-36 +0000,bbc,King 'appalled and saddened' by Bondi attack o...,https://www.bbc.com/news/articles/c8e95gkl5r3o...
3,2025-12-14-16-47-56 +0000,wapo,Live updates: 11 killed in attack on Hanukkah ...,https://www.washingtonpost.com/world/2025/12/1...
4,2025-12-14-16-41-35 +0000,nyt,Live Updates: Police Say Jewish Community Targ...,https://www.nytimes.com/live/2025/12/14/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2497/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,brown,25
5,university,21
85,trump,20
15,shooting,16
23,killed,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
137,2025-12-13-21-55-38 +0000,nypost,"At least 2 killed, 9 people injured in mass sh...",https://nypost.com/2025/12/13/us-news/active-s...,96
1,2025-12-14-16-49-56 +0000,wapo,Live updates: Police detain person of interest...,https://www.washingtonpost.com/nation/2025/12/...,89
112,2025-12-14-01-20-18 +0000,nypost,Terrified Brown University students describe ...,https://nypost.com/2025/12/13/us-news/terrifie...,88
20,2025-12-14-15-20-00 +0000,wsj,Police Detain Person of Interest in Deadly Bro...,https://www.wsj.com/us-news/police-detain-pers...,87
42,2025-12-14-12-31-01 +0000,cbc,Person of interest in custody in deadly Brown ...,https://www.cbc.ca/news/world/person-of-intere...,86


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
137,96,2025-12-13-21-55-38 +0000,nypost,"At least 2 killed, 9 people injured in mass sh...",https://nypost.com/2025/12/13/us-news/active-s...
36,66,2025-12-14-13-40-25 +0000,cbc,LIVE UPDATES | Bystander who tackled armed man...,https://www.cbc.ca/news/world/livestory/hanukk...
13,51,2025-12-14-15-50-19 +0000,nypost,Trump admits he’s not sure GOP will hold onto ...,https://nypost.com/2025/12/14/us-news/trump-ad...
153,42,2025-12-13-19-22-53 +0000,nypost,Belarus frees 123 prisoners including Nobel Pe...,https://nypost.com/2025/12/13/world-news/belar...
79,26,2025-12-14-09-42-00 +0000,wsj,"Israel Kills a Top Hamas Commander in Gaza, Mi...",https://www.wsj.com/world/middle-east/israel-k...
151,26,2025-12-13-19-32-58 +0000,nypost,"Trump tells ISIS, ‘we will retaliate’ after ‘t...",https://nypost.com/2025/12/13/us-news/presiden...
78,24,2025-12-14-10-00-00 +0000,wsj,The falling out between Rep. Marjorie Taylor G...,https://www.wsj.com/politics/policy/marjorie-t...
108,23,2025-12-14-02-50-10 +0000,nypost,Cops rescue petrified Brown students hiding fr...,https://nypost.com/2025/12/13/us-news/brown-un...
134,22,2025-12-13-22-19-55 +0000,nypost,Engine failure forces Tokyo-bound United Airli...,https://nypost.com/2025/12/13/us-news/engine-f...
122,21,2025-12-14-00-05-13 +0000,nypost,‘Broken system’ led to state’s premature relea...,https://nypost.com/2025/12/13/us-news/broken-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
